In [1]:
import pandas as pd

In [2]:
from src.parsers import SpbDnevnikPage
from src.parsers.utils import download_url

In [3]:
s = open('./raw/links/links_spbdnevnik.txt').read()

is_link = lambda word: word.startswith('https')
filter_link = lambda row: list(filter(is_link, row.split()))

links = list(map(filter_link, s.splitlines()))
links = list(filter(lambda arr: len(arr), links))
links = [x[0] for x in links]
links[:3]

['https://spbdnevnik.ru/news/2018-01-01/deputat-romanov-vruchil-invalidnuyu-kolyasku-novogo-pokoleniya-glave-fizkulturno-sportivnogo-kluba--paralimpik',
 'https://spbdnevnik.ru/news/2018-01-10/klinika--lichny-doktor--zaplatit-shtraf-za-otsutstviey-pandusov',
 'https://spbdnevnik.ru/news/2018-01-11/my-tut-ryadom']

In [4]:
from time import sleep
from tqdm import tqdm

records = []

for i, link in tqdm(enumerate(links), total=len(links)):
    sleep(0.2)
    
    try:
        source = download_url(link)
    except Exception as e:
        print(f'error downloading page {i} with err {e}')
        continue
    
    page_name = link.split('/')[-2]
    with open(f'raw/pages/{page_name}', 'w') as f:
        f.write(source)
    
    try:
        content = SpbDnevnikPage(source)
        records.append(content)
    except Exception as e:
        print(f'error parsing page {i} with err {e}')
        continue

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 713/713 [04:18<00:00,  2.75it/s]


In [5]:
def make_tuple(p) -> tuple:
    try: 
        return (p.get_header(), p.get_subtitle() + '\n' + p.get_main_text(), p.get_create_date()) 
    except: 
        print('except')
        return ('header', 'text', '2022-01-01')

tuples = [make_tuple(page) for page in records]

df = pd.DataFrame.from_records(tuples, columns=['title', 'text', 'create_dttm']).assign(link=links)
print(df.shape)
df.head()

except
(713, 4)


,title,text,create_dttm,link
0,Депутат Романов вручил инвалидную коляску ново...,"Накануне Нового года депутат Госдумы, заместит...",1 января 2018,https://spbdnevnik.ru/news/2018-01-01/deputat-...
1,"Клиника ""Личный доктор"" заплатит штраф за отсу...","В Колпинском районе медицинская клиника ООО ""Л...",10 января 2018,https://spbdnevnik.ru/news/2018-01-10/klinika-...
2,"Город услышал Антона, который тут рядом","В Петербурге будет создан центр помощи семьям,...",11 января 2018,https://spbdnevnik.ru/news/2018-01-11/my-tut-r...
3,Комитет по соцполитике прокомментировал возмож...,Председатель комитета по социальной политике А...,12 января 2018,https://spbdnevnik.ru/news/2018-01-12/komitet-...
4,Обновленный реабилитационный центр в Пушкине п...,Люди с нарушениями интеллекта нуждаются в особ...,25 января 2018,https://spbdnevnik.ru/news/2018-01-25/obnovlen...


In [6]:
for i, p in enumerate(records):
    try:
        make_tuple(p)
    except:
        print(i)

except


In [7]:
df.to_csv('./raw/articles/spbdnevnik.csv', index=False)